# Learning Agent [Compliance] - Proactive Learning Management System

Welcome to the Learning Agent [Compliance] workshop! In this notebook, you'll build an intelligent AI agent that can proactively manage employee learning compliance and send automated reminders.

## What You'll Learn

By the end of this notebook, you'll understand how to create an AI agent that can:

1. **Proactive Monitoring** - Track employee learning progress and deadlines
2. **Automated Reminders** - Send personalized reminders to employees for mandatory courses
3. **Bulk Operations** - Handle mass communications to multiple employees (1:N operations)
4. **Admin Configuration** - Allow administrators to configure custom messages and deadlines
5. **Reporting & Analytics** - Generate compliance reports per team or department

## Learning Agent [Compliance] System Overview

This Learning Agent demonstrates a **system-triggered agent** that operates autonomously to ensure organizational compliance. Key capabilities include:

- **Deadline Tracking**: Monitor course completion deadlines and identify at-risk employees
- **Personalized Communication**: Send tailored reminders based on employee roles and course types
- **Bulk Messaging**: Efficiently communicate with large groups of employees
- **Administrative Control**: Provide admins with tools to configure messaging templates and deadlines
- **Compliance Reporting**: Generate detailed reports for management oversight

## Import the Required Python Packages

Before building our Learning Agent, let's import the necessary libraries. Each import serves a specific purpose in our compliance management system:

### Core Python Libraries
- `os` - For accessing environment variables (API keys, endpoints)
- `json` - For handling JSON data in function calls and configuration
- `datetime`, `timedelta` - For date calculations and deadline management
- `typing.Annotated` - For type hints that improve function descriptions

### Azure Authentication
- `azure.identity.DefaultAzureCredential` - For Azure AD authentication
- `dotenv.load_dotenv` - For loading environment variables from .env files

### Jupyter Notebook Display
- `IPython.display` - For rich HTML output in our notebook

### Semantic Kernel Components
- `ChatCompletionAgent` - The main agent class for learning compliance management
- `ChatHistoryAgentThread` - Manages conversation history and context
- `AzureChatCompletion` - Connector for Azure OpenAI services
- `FunctionCallContent`, `FunctionResultContent`, `StreamingTextContent` - Content types for different response formats
- `kernel_function` - Decorator for creating custom agent functions (compliance tools)

In [ ]:
# Install required packages from requirements.txt
%pip install -r requirements.txt

In [ ]:
import os 
import json
from datetime import datetime, timedelta
from typing import Annotated, List, Dict, Any
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv

from IPython.display import display, HTML

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the Learning Compliance Plugin

The Learning Compliance Plugin is the heart of our system. It contains all the tools our AI agent needs to manage learning compliance effectively.

### Plugin Capabilities

Our `LearningCompliancePlugin` includes functions for:

1. **Employee Management**
   - Track employee learning records
   - Monitor course completion status
   - Identify employees approaching deadlines

2. **Communication Management**
   - Send personalized reminders to individual employees
   - Execute bulk messaging campaigns
   - Customize messages based on urgency and course type

3. **Administrative Tools**
   - Configure reminder templates and schedules
   - Set course deadlines and requirements
   - Manage department-specific policies

4. **Reporting & Analytics**
   - Generate compliance reports by team/department
   - Track completion rates and trends
   - Identify at-risk employees and courses

### Data Structure

Our plugin simulates a learning management system with:
- **Employee records** with roles, departments, and assigned courses
- **Course catalog** with requirements and deadlines
- **Message templates** for different communication scenarios
- **Compliance tracking** for reporting and analytics

### Function Design

Each `@kernel_function` is designed to:
- Provide clear descriptions for the AI agent
- Handle real-world scenarios and edge cases
- Return structured data for further processing
- Support both individual and bulk operations

In [ ]:
# Define a sample plugin for learning compliance
class LearningCompliancePlugin:
    """A Learning Compliance Plugin for managing employee training."""

    def __init__(self):
        # Simulated employee database
        self.employees = {
            "emp001": {
                "name": "John Smith",
                "email": "john.smith@company.com",
                "department": "Engineering",
                "role": "Senior Developer",
                "assigned_courses": ["SEC-101", "PY-ADVANCED", "GDPR-2024"],
                "completed_courses": ["SEC-101"],
                "course_deadlines": {
                    "PY-ADVANCED": "2024-09-15",
                    "GDPR-2024": "2024-09-30"
                }
            },
            "emp002": {
                "name": "Sarah Johnson",
                "email": "sarah.johnson@company.com",
                "department": "Marketing",
                "role": "Marketing Manager",
                "assigned_courses": ["GDPR-2024", "LEAD-101"],
                "completed_courses": [],
                "course_deadlines": {
                    "GDPR-2024": "2024-09-30",
                    "LEAD-101": "2024-10-15"
                }
            },
            "emp003": {
                "name": "Michael Chen",
                "email": "michael.chen@company.com",
                "department": "Engineering",
                "role": "Junior Developer",
                "assigned_courses": ["SEC-101", "PY-BASICS", "GDPR-2024"],
                "completed_courses": ["PY-BASICS"],
                "course_deadlines": {
                    "SEC-101": "2024-09-10",
                    "GDPR-2024": "2024-09-30"
                }
            },
            "emp004": {
                "name": "Lisa Wang",
                "email": "lisa.wang@company.com",
                "department": "HR",
                "role": "HR Specialist",
                "assigned_courses": ["GDPR-2024", "HR-COMP", "LEAD-101"],
                "completed_courses": ["HR-COMP"],
                "course_deadlines": {
                    "GDPR-2024": "2024-09-30",
                    "LEAD-101": "2024-10-15"
                }
            }
        }

        # Course catalog
        self.courses = {
            "SEC-101": {"name": "Security Fundamentals", "duration": "4 hours", "mandatory": True},
            "PY-ADVANCED": {"name": "Advanced Python Programming", "duration": "8 hours", "mandatory": False},
            "PY-BASICS": {"name": "Python Basics", "duration": "6 hours", "mandatory": False},
            "GDPR-2024": {"name": "GDPR Compliance 2024", "duration": "2 hours", "mandatory": True},
            "LEAD-101": {"name": "Leadership Fundamentals", "duration": "6 hours", "mandatory": False},
            "HR-COMP": {"name": "HR Compliance Training", "duration": "4 hours", "mandatory": True}
        }

        # Message templates
        self.message_templates = {
            "urgent": "🚨 URGENT: Your '{course_name}' course is due in {days_remaining} days. Please complete it immediately to maintain compliance.",
            "reminder": "📚 Reminder: Your '{course_name}' course is due on {deadline}. Please complete it at your earliest convenience.",
            "overdue": "⚠️ OVERDUE: Your '{course_name}' course was due on {deadline}. Please complete it immediately to avoid compliance issues.",
            "bulk_deadline": "📅 Department Reminder: The following courses have upcoming deadlines. Please ensure completion by the specified dates.",
            "welcome": "👋 Welcome! You have been assigned new learning courses. Please review your assignments and complete them by the specified deadlines."
        }

    @kernel_function(description="Get employees who have upcoming course deadlines within specified days.")
    def get_employees_with_upcoming_deadlines(
        self, 
        days_ahead: Annotated[int, "Number of days to look ahead for deadlines"] = 7
    ) -> Annotated[str, "List of employees with upcoming deadlines"]:
        """Find employees with courses due within the specified number of days."""
        
        current_date = datetime.now()
        target_date = current_date + timedelta(days=days_ahead)
        
        upcoming_deadlines = []
        
        for emp_id, employee in self.employees.items():
            pending_courses = []
            for course_id in employee["assigned_courses"]:
                if course_id not in employee["completed_courses"]:
                    deadline_str = employee["course_deadlines"].get(course_id)
                    if deadline_str:
                        deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                        if current_date <= deadline <= target_date:
                            days_remaining = (deadline - current_date).days
                            pending_courses.append({
                                "course_id": course_id,
                                "course_name": self.courses[course_id]["name"],
                                "deadline": deadline_str,
                                "days_remaining": days_remaining,
                                "is_mandatory": self.courses[course_id]["mandatory"]
                            })
            
            if pending_courses:
                upcoming_deadlines.append({
                    "employee_id": emp_id,
                    "name": employee["name"],
                    "email": employee["email"],
                    "department": employee["department"],
                    "pending_courses": pending_courses
                })
        
        return json.dumps(upcoming_deadlines, indent=2)

    @kernel_function(description="Send a personalized reminder to a specific employee about their pending courses.")
    def send_reminder_to_employee(
        self, 
        employee_id: Annotated[str, "Employee ID to send reminder to"],
        message_type: Annotated[str, "Type of message: urgent, reminder, overdue, welcome"] = "reminder"
    ) -> Annotated[str, "Confirmation of reminder sent with message content"]:
        """Send a personalized learning reminder to an employee."""
        
        if employee_id not in self.employees:
            return f"Error: Employee {employee_id} not found."
        
        employee = self.employees[employee_id]
        pending_courses = []
        
        # Find pending courses
        for course_id in employee["assigned_courses"]:
            if course_id not in employee["completed_courses"]:
                deadline_str = employee["course_deadlines"].get(course_id)
                if deadline_str:
                    deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                    days_remaining = (deadline - datetime.now()).days
                    pending_courses.append({
                        "course_id": course_id,
                        "course_name": self.courses[course_id]["name"],
                        "deadline": deadline_str,
                        "days_remaining": days_remaining
                    })
        
        if not pending_courses:
            return f"No pending courses found for {employee['name']}."
        
        # Generate personalized message
        messages = []
        template = self.message_templates.get(message_type, self.message_templates["reminder"])
        
        for course in pending_courses:
            message = template.format(
                course_name=course["course_name"],
                deadline=course["deadline"],
                days_remaining=course["days_remaining"]
            )
            messages.append(message)
        
        result = {
            "recipient": {
                "name": employee["name"],
                "email": employee["email"],
                "department": employee["department"]
            },
            "message_type": message_type,
            "messages_sent": messages,
            "courses_count": len(pending_courses),
            "status": "sent"
        }
        
        return json.dumps(result, indent=2)

    @kernel_function(description="Send bulk reminders to multiple employees in a department or with specific criteria.")
    def send_bulk_reminders(
        self, 
        department: Annotated[str, "Department name to send reminders to (optional)"] = "",
        days_ahead: Annotated[int, "Number of days to look ahead for deadlines"] = 7,
        message_type: Annotated[str, "Type of message: urgent, reminder, bulk_deadline"] = "bulk_deadline"
    ) -> Annotated[str, "Summary of bulk reminders sent"]:
        """Send reminders to multiple employees based on department or deadline criteria."""
        
        current_date = datetime.now()
        target_date = current_date + timedelta(days=days_ahead)
        
        eligible_employees = []
        
        # Filter employees by department if specified
        for emp_id, employee in self.employees.items():
            if department and employee["department"].lower() != department.lower():
                continue
                
            # Check for pending courses with upcoming deadlines
            pending_courses = []
            for course_id in employee["assigned_courses"]:
                if course_id not in employee["completed_courses"]:
                    deadline_str = employee["course_deadlines"].get(course_id)
                    if deadline_str:
                        deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                        if current_date <= deadline <= target_date:
                            pending_courses.append({
                                "course_name": self.courses[course_id]["name"],
                                "deadline": deadline_str
                            })
            
            if pending_courses:
                eligible_employees.append({
                    "employee_id": emp_id,
                    "name": employee["name"],
                    "email": employee["email"],
                    "department": employee["department"],
                    "pending_courses": pending_courses
                })
        
        # Generate bulk message results
        bulk_results = {
            "campaign_info": {
                "target_department": department or "All departments",
                "message_type": message_type,
                "days_ahead": days_ahead,
                "date_sent": current_date.strftime("%Y-%m-%d %H:%M:%S")
            },
            "recipients": eligible_employees,
            "summary": {
                "total_recipients": len(eligible_employees),
                "departments_affected": list(set([emp["department"] for emp in eligible_employees])),
                "total_pending_courses": sum([len(emp["pending_courses"]) for emp in eligible_employees])
            }
        }
        
        return json.dumps(bulk_results, indent=2)

    @kernel_function(description="Configure message templates and deadline settings for the compliance system.")
    def configure_compliance_settings(
        self, 
        setting_type: Annotated[str, "Type of setting: message_template, deadline, course_assignment"],
        setting_key: Annotated[str, "Specific setting key to configure"],
        setting_value: Annotated[str, "New value for the setting"]
    ) -> Annotated[str, "Confirmation of configuration change"]:
        """Allow administrators to configure compliance system settings."""
        
        if setting_type == "message_template":
            if setting_key in self.message_templates:
                old_value = self.message_templates[setting_key]
                self.message_templates[setting_key] = setting_value
                
                result = {
                    "setting_type": setting_type,
                    "setting_key": setting_key,
                    "old_value": old_value,
                    "new_value": setting_value,
                    "status": "updated",
                    "message": f"Message template '{setting_key}' has been updated successfully."
                }
            else:
                result = {
                    "setting_type": setting_type,
                    "setting_key": setting_key,
                    "status": "error",
                    "message": f"Message template '{setting_key}' not found. Available templates: {list(self.message_templates.keys())}"
                }
        else:
            result = {
                "setting_type": setting_type,
                "status": "not_implemented",
                "message": f"Configuration for {setting_type} is not yet implemented in this demo."
            }
        
        return json.dumps(result, indent=2)

    @kernel_function(description="Generate compliance reports by department, team, or individual employee.")
    def generate_compliance_report(
        self, 
        report_type: Annotated[str, "Type of report: department, individual, overview"],
        target: Annotated[str, "Target for report (department name or employee ID)"] = ""
    ) -> Annotated[str, "Detailed compliance report"]:
        """Generate comprehensive compliance reports for management oversight."""
        
        current_date = datetime.now()
        
        if report_type == "department":
            # Department-specific report
            department_employees = {
                emp_id: emp for emp_id, emp in self.employees.items() 
                if emp["department"].lower() == target.lower()
            }
            
            if not department_employees:
                return f"No employees found in department: {target}"
            
            dept_stats = {
                "department": target,
                "total_employees": len(department_employees),
                "compliance_summary": [],
                "at_risk_employees": [],
                "completed_courses_total": 0,
                "pending_courses_total": 0
            }
            
            for emp_id, employee in department_employees.items():
                completed = len(employee["completed_courses"])
                pending = len(employee["assigned_courses"]) - completed
                
                dept_stats["completed_courses_total"] += completed
                dept_stats["pending_courses_total"] += pending
                
                # Check for overdue courses
                overdue_courses = []
                for course_id in employee["assigned_courses"]:
                    if course_id not in employee["completed_courses"]:
                        deadline_str = employee["course_deadlines"].get(course_id)
                        if deadline_str:
                            deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                            if deadline < current_date:
                                overdue_courses.append(self.courses[course_id]["name"])
                
                employee_summary = {
                    "employee_id": emp_id,
                    "name": employee["name"],
                    "completed_courses": completed,
                    "pending_courses": pending,
                    "overdue_courses": overdue_courses,
                    "compliance_status": "At Risk" if overdue_courses else "On Track"
                }
                
                dept_stats["compliance_summary"].append(employee_summary)
                
                if overdue_courses:
                    dept_stats["at_risk_employees"].append(employee_summary)
            
            return json.dumps(dept_stats, indent=2)
        
        elif report_type == "individual":
            # Individual employee report
            if target not in self.employees:
                return f"Employee {target} not found."
            
            employee = self.employees[target]
            
            course_details = []
            for course_id in employee["assigned_courses"]:
                status = "Completed" if course_id in employee["completed_courses"] else "Pending"
                deadline_str = employee["course_deadlines"].get(course_id, "No deadline")
                
                course_info = {
                    "course_id": course_id,
                    "course_name": self.courses[course_id]["name"],
                    "status": status,
                    "deadline": deadline_str,
                    "mandatory": self.courses[course_id]["mandatory"]
                }
                
                if status == "Pending" and deadline_str != "No deadline":
                    deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                    if deadline < current_date:
                        course_info["status"] = "Overdue"
                    else:
                        days_remaining = (deadline - current_date).days
                        course_info["days_remaining"] = days_remaining
                
                course_details.append(course_info)
            
            individual_report = {
                "employee_info": {
                    "employee_id": target,
                    "name": employee["name"],
                    "email": employee["email"],
                    "department": employee["department"],
                    "role": employee["role"]
                },
                "course_summary": {
                    "total_assigned": len(employee["assigned_courses"]),
                    "completed": len(employee["completed_courses"]),
                    "pending": len(employee["assigned_courses"]) - len(employee["completed_courses"])
                },
                "course_details": course_details
            }
            
            return json.dumps(individual_report, indent=2)
        
        else:
            # Overview report
            overview_stats = {
                "organization_overview": {
                    "total_employees": len(self.employees),
                    "departments": list(set([emp["department"] for emp in self.employees.values()])),
                    "total_courses": len(self.courses),
                    "mandatory_courses": len([c for c in self.courses.values() if c["mandatory"]])
                },
                "department_breakdown": {},
                "at_risk_summary": {
                    "employees_with_overdue": 0,
                    "total_overdue_courses": 0
                }
            }
            
            # Calculate department breakdowns
            for emp_id, employee in self.employees.items():
                dept = employee["department"]
                if dept not in overview_stats["department_breakdown"]:
                    overview_stats["department_breakdown"][dept] = {
                        "employee_count": 0,
                        "total_completed": 0,
                        "total_pending": 0,
                        "employees_at_risk": 0
                    }
                
                overview_stats["department_breakdown"][dept]["employee_count"] += 1
                overview_stats["department_breakdown"][dept]["total_completed"] += len(employee["completed_courses"])
                overview_stats["department_breakdown"][dept]["total_pending"] += (
                    len(employee["assigned_courses"]) - len(employee["completed_courses"])
                )
                
                # Check for overdue courses
                has_overdue = False
                for course_id in employee["assigned_courses"]:
                    if course_id not in employee["completed_courses"]:
                        deadline_str = employee["course_deadlines"].get(course_id)
                        if deadline_str:
                            deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
                            if deadline < current_date:
                                has_overdue = True
                                overview_stats["at_risk_summary"]["total_overdue_courses"] += 1
                
                if has_overdue:
                    overview_stats["department_breakdown"][dept]["employees_at_risk"] += 1
                    overview_stats["at_risk_summary"]["employees_with_overdue"] += 1
            
            return json.dumps(overview_stats, indent=2)

## Creating the AI Service Connection

Just like in previous workshops, we need to establish a connection to our AI service. Our Learning Agent [Compliance] system will use Azure OpenAI to provide intelligent analysis and natural language interactions.

### Why AI for Compliance Management?

The AI component adds significant value to traditional compliance systems:

- **Natural Language Queries**: Administrators can ask questions like "Which Engineering employees are behind on security training?"
- **Intelligent Prioritization**: The agent can identify the most critical compliance issues and suggest actions
- **Personalized Communication**: Generate contextual, empathetic reminder messages
- **Trend Analysis**: Interpret compliance data and provide insights about patterns and risks
- **Automated Decision Making**: Determine appropriate reminder schedules and escalation paths

### Configuration for Compliance Scenarios

Our Azure OpenAI configuration is optimized for:
- **Reliable Performance**: Consistent response times for automated operations
- **Security**: Protecting sensitive employee and compliance data
- **Scalability**: Handling bulk operations across large organizations
- **Integration**: Working seamlessly with existing HR and LMS systems

In [ ]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# credential = DefaultAzureCredential()
# 
# def get_azure_ad_token():
#     """Function to get Azure AD token for OpenAI."""
#     token = credential.get_token("https://cognitiveservices.azure.com/.default")
#     return token.token
# 
# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()
# )

## Creating the Learning Compliance Agent

Now we'll create our **Learning Agent [Compliance]** using the `ChatCompletionAgent` class. This agent embodies the role of a proactive compliance manager that can operate autonomously while providing human administrators with powerful tools and insights.

### Agent Personality and Behavior

Our Learning Compliance Agent is designed with specific characteristics:

- **Proactive**: Identifies issues before they become critical
- **Professional**: Maintains appropriate tone for workplace communications
- **Empathetic**: Understands that employees have varying schedules and constraints
- **Data-Driven**: Makes decisions based on actual compliance data and deadlines
- **Actionable**: Always provides clear next steps and recommendations

### Key Instruction Patterns for Compliance

The agent's instructions demonstrate several important patterns for system-triggered agents:

1. **Role Clarity**: "You are a Learning Compliance Manager AI Agent"
2. **Operational Guidelines**: When to escalate, how to prioritize, communication standards
3. **Data Interpretation**: How to analyze compliance reports and identify risks
4. **Communication Standards**: Tone, urgency levels, and personalization approaches
5. **Automation Logic**: When to trigger reminders, how to handle bulk operations

### System-Triggered vs User-Initiated

Unlike conversational agents, our Learning Compliance Agent is designed to:
- Operate on schedules and triggers (daily compliance checks)
- Take autonomous actions (sending reminders)
- Generate reports proactively (weekly department summaries)
- Escalate issues when thresholds are met (overdue mandatory training)

💡 **Pro Tip**: System-triggered agents require more detailed instructions about decision-making criteria and escalation procedures since they operate with less human oversight.

In [ ]:
COMPLIANCE_AGENT_INSTRUCTIONS = """You are a Learning Compliance Manager AI Agent responsible for ensuring organizational learning compliance and proactively managing employee training requirements.

Your primary responsibilities include:
1. **Proactive Monitoring**: Continuously track employee learning progress and identify those approaching or missing deadlines
2. **Automated Communication**: Send personalized reminders and bulk notifications based on urgency and course requirements
3. **Administrative Support**: Help admins configure messaging templates, deadlines, and compliance policies
4. **Reporting & Analytics**: Generate detailed compliance reports for management oversight and decision-making

**Communication Guidelines:**
- For urgent/overdue courses: Use professional but firm language emphasizing compliance requirements
- For general reminders: Be encouraging and supportive while maintaining professionalism  
- For bulk communications: Focus on clarity and provide actionable next steps
- Always personalize messages when possible (employee name, specific courses, deadlines)

**Operational Priorities:**
1. **Mandatory courses** take precedence over optional training
2. **Overdue courses** require immediate attention and escalation
3. **Courses due within 3 days** should trigger urgent reminders
4. **Department-wide issues** may require bulk communication strategies

**Reporting Standards:**
- Provide clear metrics: completion rates, at-risk employees, overdue counts
- Identify trends and patterns in compliance data
- Suggest actionable recommendations for improvement
- Highlight both successes and areas of concern

**Decision-Making Criteria:**
- Send individual reminders for courses due within 7 days
- Trigger urgent alerts for courses due within 3 days or overdue
- Generate bulk reminders for department-wide deadline approaches
- Escalate to management when multiple employees in critical roles are non-compliant

When users ask about compliance status, always check current data and provide specific, actionable insights. Be proactive in suggesting next steps and preventive measures.
"""

compliance_agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[LearningCompliancePlugin()],
    name="LearningComplianceAgent",
    instructions=COMPLIANCE_AGENT_INSTRUCTIONS,
)

## Running the Learning Compliance Agent

Let's test our Learning Compliance Agent with various scenarios that demonstrate its capabilities. Our test cases cover the full spectrum of compliance management operations.

### Test Scenarios Explained

Our test inputs demonstrate different aspects of the compliance system:

#### 1. **Proactive Monitoring**
- "Check which employees have upcoming course deadlines in the next week"
- Tests the agent's ability to identify at-risk employees proactively

#### 2. **Individual Intervention**
- "Send a reminder to employee emp001 about their pending courses"
- Demonstrates personalized communication capabilities

#### 3. **Bulk Operations**
- "Send bulk reminders to all Engineering department employees"
- Shows how the system handles department-wide communications

#### 4. **Administrative Configuration**
- "Update the urgent message template to be more supportive"
- Tests the system's flexibility and administrative controls

#### 5. **Reporting & Analytics**
- "Generate a compliance report for the Engineering department"
- Demonstrates comprehensive reporting capabilities

### What to Observe

As you run these scenarios, pay attention to:

- **Data Analysis**: How the agent interprets compliance data and identifies priorities
- **Communication Style**: The tone and personalization in reminder messages
- **Operational Logic**: When the agent chooses individual vs bulk communication
- **Report Quality**: The depth and actionability of generated compliance reports
- **Administrative Features**: How configuration changes affect system behavior

### Real-World Integration

In a production environment, this agent would typically:
- Run on automated schedules (daily, weekly)
- Integrate with existing HR and LMS systems
- Send actual emails through corporate communication systems
- Generate reports for management dashboards
- Trigger escalations based on compliance thresholds

💡 **Experiment**: Try asking the agent follow-up questions about the compliance data or requesting different types of reports to see how it adapts its responses.

In [ ]:
# Test scenarios to demonstrate Learning Compliance Agent capabilities
user_inputs = [
    "Check which employees have upcoming course deadlines in the next week.",
    "Send a reminder to employee emp001 about their pending courses.",
    "Send bulk reminders to all Engineering department employees about upcoming deadlines.",
    "Update the urgent message template to be more supportive and less intimidating.",
    "Generate a compliance report for the Engineering department.",
    "Show me an overview of our organization's learning compliance status."
]

async def main():
    # Initialize conversation thread (maintains context across operations)
    thread: ChatHistoryAgentThread | None = None

    # Process each compliance operation sequentially
    for user_input in user_inputs:
        # Start building HTML output for display
        html_output = (
            f"<div style='margin-bottom:10px; border-left: 4px solid #007acc; padding-left: 15px;'>"
            f"<div style='font-weight:bold; color: #007acc;'>🎯 Compliance Operation:</div>"
            f"<div style='margin-left:20px; font-style: italic;'>{user_input}</div></div>"
        )

        # Initialize response tracking variables
        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer for streaming function calls
        current_function_name = None
        argument_buffer = ""

        # Stream agent response in real-time
        async for response in compliance_agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            # Update thread with conversation history
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            # Process each content item in the response
            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    # Capture function name when agent calls a compliance tool
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate function arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Process completed function call
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            # Pretty-print JSON arguments if possible
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args, indent=2)
                        except Exception:
                            pass  # Keep as raw string if not valid JSON

                        function_calls.append(f"🔧 Executing: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    # Capture function result
                    function_calls.append(f"\n📊 System Response:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    # Collect agent's analysis and recommendations
                    full_response.append(item.text)

        # Add collapsible function calls section to output
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#ff6b35;'>🛠️ System Operations (click to expand)</summary>"
                "<div style='margin:10px; padding:15px; background-color:#f8f9fa; "
                "border:1px solid #e9ecef; border-radius:6px; white-space:pre-wrap; font-size:13px; color:#495057;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        # Add agent response to output
        html_output += (
            "<div style='margin-bottom:20px; padding: 15px; background-color:#f0f8ff; border-radius: 6px;'>"
            f"<div style='font-weight:bold; color:#1a5490;'>🤖 {agent_name or 'Learning Compliance Agent'} Analysis:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap; line-height: 1.6;'>{''.join(full_response)}</div></div>"
            "<hr style='border: none; height: 2px; background: linear-gradient(to right, #007acc, transparent);'>"
        )

        # Display the formatted compliance operation results
        display(HTML(html_output))

# Run the Learning Compliance Agent scenarios
await main()

## 🎉 Congratulations!

You've successfully built a comprehensive Learning Agent [Compliance] system! Let's review what you've accomplished and learned:

### What You Built

✅ **Proactive Monitoring System** - Automated tracking of employee learning progress and deadlines  
✅ **Intelligent Communication Engine** - Personalized and bulk reminder capabilities  
✅ **Administrative Control Panel** - Configurable message templates and compliance settings  
✅ **Comprehensive Reporting** - Department and individual compliance analytics  
✅ **System-Triggered Operations** - Autonomous agent that can operate without constant human intervention  
✅ **Multi-Modal Communication** - Individual reminders, bulk campaigns, and administrative notifications  

### Next Steps for Production Deployment

To deploy this system in a real organization, consider:

1. **Integration Points**
   - Connect to existing HR Information Systems (HRIS)
   - Integrate with Learning Management Systems (LMS)
   - Link to email/communication platforms (Outlook, Teams, Slack)

2. **Data Security & Privacy**
   - Implement role-based access controls
   - Ensure GDPR/privacy compliance for employee data
   - Add audit logging for all compliance actions

3. **Scalability Enhancements**
   - Database integration for large employee populations
   - Batch processing for high-volume operations
   - Caching strategies for frequently accessed data

4. **Advanced Features**
   - Machine learning for completion prediction
   - Integration with calendar systems for optimal reminder timing
   - Automated escalation to managers for critical compliance issues

### Experiment and Extend

Try these enhancements to deepen your understanding:

1. **Add new compliance rules** (e.g., course prerequisites, role-based requirements)
2. **Implement escalation workflows** (manager notifications, HR alerts)
3. **Create dashboard integration** for real-time compliance monitoring
4. **Add course recommendation features** based on role and career development
5. **Implement compliance scoring** and gamification elements